In [ ]:
#Loading it up
reviews_train = []
reviews_test = []

for line in open('aclImdb/movie_data/full_train.txt'):
    reviews_train.append(line.strip().lower())
    
for line in open('aclImdb/movie_data/full_test.txt'):
    reviews_test.append(line.strip().lower())
    

In [ ]:
engStopWords = ['a', 'the', 'in', 'of', 'at', 'is']
def removeStopWords(reviewList):
    cleanReviews = []
    for review in reviewList:
        cleanReviews.append(' '.join(word for word in review.split() if word not in engStopWords))
    
    return cleanReviews

reviews_train = removeStopWords(reviews_train)
reviews_test = removeStopWords(reviews_test)


In [ ]:
# Stemming -> https://www.nltk.org/howto/stem.html
from nltk.stem import SnowballStemmer

def stemWords(reviewList):
    stemReviews = []
    for review in reviewList:
        stemReviews.append(' '.join(SnowballStemmer("porter").stem(word) for word in review.split()))
        # "porter" gives more accuracy than "english"
    
    return stemReviews

reviews_train = stemWords(reviews_train)
reviews_test = stemWords(reviews_test)

In [50]:
# Ngram Count Vectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

def getVectorizer(type):
    if type == 'cv':
        return CountVectorizer(binary=True, ngram_range=(1,2))
    elif type == 'tfidf':
        return TfidfVectorizer()
    elif type == 'cv-grp':
        return CountVectorizer(binary=False, ngram_range=(1,2))

vec = getVectorizer('cv')
vec.fit(reviews_train)

X = vec.transform(reviews_train)
X_test = vec.transform(reviews_test)


In [ ]:
# LOGISTIC REGRESSION - Finding optimal value for c
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

target = [1 if i<12500 else 0 for i in range(25000)]

X_train, X_val, Y_train, Y_val = train_test_split(X, target, train_size=0.75)

for c in [0.01, 0.05, 0.1, 0.5, 1]:
    lr = LogisticRegression(C=c)
    lr.fit(X_train, Y_train)

    print("Accuracy score for c = %s is %s"%(c, accuracy_score(Y_val, lr.predict(X_val))))


In [47]:
# Moving on with C = 0.5

lr = LogisticRegression(C=0.5)
lr.fit(X, target)

print("Final accuracy score for c = %s is %s"%(0.5, accuracy_score(target, lr.predict(X_test))))



Final accuracy score for c = 0.5 is 0.89776
